### MinIO explained

In [70]:
import boto3
import os

# These come from your docker-compose env vars
aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"]     # "admin"
aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"]  # "password"
aws_region = os.environ["AWS_REGION"] # us-east-1

# Mocked S3 client that connects to local MinIO
s3 = boto3.client(
    "s3",
    endpoint_url="http://minio:9000",  # Local MinIO service
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# Create a bucket
s3.create_bucket(Bucket="poc")

# Create a blob (upload a file)
s3.put_object(Bucket="poc", Key="demo.txt", Body=b"Hello, Iceberg!")
print()

In [71]:
# List all the blobs

response = s3.list_objects_v2(Bucket="poc")
for obj in response.get("Contents", []):
    print(f"Found object: {obj['Key']}")

Found object: demo.txt


In [72]:
# Read the blob

response = s3.get_object(Bucket="poc", Key="demo.txt")
print(response["Body"].read().decode())

Hello, Iceberg!


In [73]:
# Clean it up

# Delete blob
s3.delete_object(Bucket="poc", Key="demo.txt")

# Delete bucket
s3.delete_bucket(Bucket="poc")
print()

### Iceberg time!

In [74]:
from pyiceberg.catalog import load_rest

catalog = load_rest(
    name="rest",
    conf = {
        "uri": "http://rest:8181/",
        "s3.endpoint": "http://minio:9000",
        "s3.access-key": aws_access_key_id,
        "s3.secret-key": aws_secret_access_key
    }
)

In [75]:
from pyiceberg.exceptions import NamespaceAlreadyExistsError

namespace = "rideshare"

try:
    catalog.create_namespace(namespace)
except NamespaceAlreadyExistsError:
    pass  # It's fine if it already exists

In [76]:
namespaces = catalog.list_namespaces()
print("Namespaces:", namespaces)

Namespaces: [('rideshare',)]


In [77]:
# First, imports
from pyiceberg.schema import Schema
from pyiceberg.types import (
    NestedField, StringType, DateType, DoubleType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField

In [78]:
# ❗ Iceberg requires all fields to have stable, explicit IDs.
# This is critical for schema evolution and tracking changes over time.
# That's why we use NestedField() — each field has:
# - field_id: required, stable numeric ID
# - name: field name
# - field_type: Iceberg data type
# - required: whether the field is NOT NULL

rides_schema = Schema(
    NestedField(field_id=1, name="ride_id", field_type=StringType(), required=True),
    NestedField(field_id=2, name="driver_id", field_type=StringType()),
    NestedField(field_id=3, name="customer_id", field_type=StringType()),
    NestedField(field_id=4, name="pickup_time", field_type=DateType()),
    NestedField(field_id=5, name="dropoff_time", field_type=DateType()),
    NestedField(field_id=6, name="fare", field_type=DoubleType()),
    NestedField(field_id=7, name="pickup_location", field_type=StringType()),
    NestedField(field_id=8, name="dropoff_location", field_type=StringType()),
    NestedField(field_id=9, name="status", field_type=StringType()),
    NestedField(field_id=10, name="pickup_day", field_type=DateType())  # 👈 NEW column for partitioning
)

In [79]:
rides_partition_spec = PartitionSpec(
    fields=[
        PartitionField(
            source_id=10,      # 👈 pickup_day's field ID
            field_id=1000,     # unique ID for this partition
            transform="identity",
            name="pickup_day"
        )
    ]
)

In [80]:
# catalog.drop_table(f"{namespace}.rides")

catalog.create_table(
    identifier=f"{namespace}.rides",
    schema=rides_schema,
    partition_spec=rides_partition_spec
)

rides(
  1: ride_id: required string,
  2: driver_id: optional string,
  3: customer_id: optional string,
  4: pickup_time: optional date,
  5: dropoff_time: optional date,
  6: fare: optional double,
  7: pickup_location: optional string,
  8: dropoff_location: optional string,
  9: status: optional string,
  10: pickup_day: optional date
),
partition by: [pickup_day],
sort order: [],
snapshot: null

In [98]:
import json

response = s3.list_objects_v2(Bucket="warehouse", Prefix="rideshare/rides/metadata/")

for obj in response.get("Contents", []):
    obj_name = obj["Key"]
    print(f"Found file: {obj_name}")
    
    if obj_name.endswith("metadata.json"):
        response = s3.get_object(Bucket="warehouse", Key=obj_name)
        content = response["Body"].read().decode()
        metadata = json.loads(content)
        
        print("\n--- Parsed metadata.json ---\n")
        print(json.dumps(metadata, indent=2))

In [82]:
# 1. Generate sample data
import pandas as pd
from uuid import uuid4
from datetime import datetime

# Sample data
df = pd.DataFrame([
    {
        "ride_id": str(uuid4()),  # use string UUID
        "driver_id": "driver_001",
        "customer_id": "cust_001",
        "pickup_time": datetime(2025, 2, 1, 8, 30).date(),
        "dropoff_time": datetime(2025, 2, 1, 8, 50).date(),
        "fare": 23.5,
        "pickup_location": "Downtown",
        "dropoff_location": "Airport",
        "status": "completed"
    }
])

df["pickup_day"] = df["pickup_time"]

In [83]:
# 2. Convert it to Arrow table.
import pyarrow as pa

# Arrow schema
schema = pa.schema([
    pa.field("ride_id", pa.string(), nullable=False),
    ("driver_id", pa.string()),
    ("customer_id", pa.string()),
    ("pickup_time", pa.date32()),
    ("dropoff_time", pa.date32()),
    ("fare", pa.float64()),
    ("pickup_location", pa.string()),
    ("dropoff_location", pa.string()),
    ("status", pa.string()),
    ("pickup_day", pa.date32())
])

# Convert to Arrow Table
table = pa.Table.from_pandas(df, schema=schema, preserve_index=False)

In [84]:
# 3. Register it with the Iceberg table
table_identifier = "rideshare.rides"

# Just load the table and append the data
iceberg_table = catalog.load_table(table_identifier)
iceberg_table.append(table)

print(f"📌 Data appended to Iceberg table: {table_identifier}")

📌 Data appended to Iceberg table: rideshare.rides


#### Let's try to add new field in pyiceberg, without changing the schema

In [87]:
# 1. Add new field
df["tip_amount"] = 3.25

# 2. Update the Arrow schema to include the new column:
schema_tip = pa.schema([
    pa.field("ride_id", pa.string(), nullable=False),
    pa.field("driver_id", pa.string()),
    pa.field("customer_id", pa.string()),
    pa.field("pickup_time", pa.date32()),
    pa.field("dropoff_time", pa.date32()),
    pa.field("fare", pa.float64()),
    pa.field("pickup_location", pa.string()),
    pa.field("dropoff_location", pa.string()),
    pa.field("status", pa.string()),
    pa.field("pickup_day", pa.date32()),
    pa.field("tip_amount", pa.float64())  # New field
])

table = pa.Table.from_pandas(df, schema=schema_tip, preserve_index=False)

In [86]:
try:
    iceberg_table = catalog.load_table("rideshare.rides")
    iceberg_table.append(table)
except ValueError:
    print("Cannot append data with different schema!")

Cannot append data with different schema!


In [88]:
# This function returns a connected cursor and conn so you can reuse it flexibly.
import mysql.connector
from contextlib import contextmanager

@contextmanager
def starrocks_cursor(host="starrocks-fe", port=9030, user="root", password=""):
    conn = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password
    )
    cursor = conn.cursor()
    try:
        yield cursor
        conn.commit()
    finally:
        cursor.close()
        conn.close()

In [89]:
# Test connection
with starrocks_cursor() as cursor:
    cursor.execute("SELECT 1;")
    print("Result:", cursor.fetchall())

Result: [(1,)]


In [92]:
# Register catalog

sql_query = f"""
CREATE EXTERNAL CATALOG 'demo'
COMMENT "External catalog to Apache Iceberg on MinIO"
PROPERTIES
(
  "type"="iceberg",  -- defines this as an iceberg catalog
  "iceberg.catalog.type"="rest",  -- uses the rest catalog backend
  "iceberg.catalog.uri"="http://iceberg-rest:8181",  -- rest catalog endpoint
  "iceberg.catalog.warehouse"="warehouse",  -- root path in s3/minio
  "aws.s3.access_key"="{os.environ["AWS_ACCESS_KEY_ID"]}",  -- minio access key
  "aws.s3.secret_key"="{os.environ["AWS_SECRET_ACCESS_KEY"]}",  -- minio secret key
  "aws.s3.endpoint"="http://minio:9000",  -- minio api endpoint
  "aws.s3.enable_path_style_access"="true",  -- required for minio compatibility
  "client.factory"="com.starrocks.connector.iceberg.IcebergAwsClientFactory"  -- tells starrocks to use aws-style s3 client
);
"""

with starrocks_cursor() as cursor:
    cursor.execute(sql_query)
    print("✅ Iceberg catalog `demo` registered.")

✅ Iceberg catalog `demo` registered.


In [93]:
with starrocks_cursor() as cursor:
    cursor.execute("SHOW CATALOGS;")
    print(cursor.fetchall())

[('default_catalog', 'Internal', "An internal catalog contains this cluster's self-managed tables."), ('demo', 'Iceberg', 'External catalog to Apache Iceberg on MinIO')]


In [91]:
# with starrocks_cursor() as cursor:
#     cursor.execute("drop catalog demo;")
#     print("🗑️ Catalog 'demo' deleted.")

🗑️ Catalog 'demo' deleted.


In [94]:
with starrocks_cursor() as cursor:
    cursor.execute("SELECT * FROM demo.rideshare.rides;")
    print(cursor.fetchall())

[('d569cc54-ea35-4e78-8cb9-411cbce02a85', 'driver_001', 'cust_001', datetime.date(2025, 2, 1), datetime.date(2025, 2, 1), 23.5, 'Downtown', 'Airport', 'completed', datetime.date(2025, 2, 1))]


In [95]:
# Insert new record
with starrocks_cursor() as cursor:
    cursor.execute("""
        INSERT INTO demo.rideshare.rides 
        VALUES (
            '00000000-0000-0000-0000-000000000999',  -- ride_id
            'driver_demo',                          -- driver_id
            'customer_demo',                        -- customer_id
            '2025-06-01 10:00:00',                  -- pickup_time
            '2025-06-01 10:15:00',                  -- dropoff_time
            12.50,                                  -- fare
            'Downtown',                             -- pickup_location
            'Uptown',                               -- dropoff_location
            'completed',                            -- status
            '2025-06-01'                            -- pickup_day
        );
    """)
    print("✅ Row inserted.")

✅ Row inserted.


In [96]:
with starrocks_cursor() as cursor:
    cursor.execute("SELECT * FROM demo.rideshare.rides;")
    print(cursor.fetchall())

[('00000000-0000-0000-0000-000000000999', 'driver_demo', 'customer_demo', datetime.date(2025, 6, 1), datetime.date(2025, 6, 1), 12.5, 'Downtown', 'Uptown', 'completed', datetime.date(2025, 6, 1)), ('d569cc54-ea35-4e78-8cb9-411cbce02a85', 'driver_001', 'cust_001', datetime.date(2025, 2, 1), datetime.date(2025, 2, 1), 23.5, 'Downtown', 'Airport', 'completed', datetime.date(2025, 2, 1))]


#### Let's try to add new record in pyiceberg

In [49]:
from datetime import datetime
from uuid import uuid4

df = df.drop(columns=["tip_amount"])

# New record with tip_amount
new_record = {
    "ride_id": str(uuid4()),
    "driver_id": "driver_002",
    "customer_id": "cust_002",
    "pickup_time": datetime(2025, 2, 2, 9, 0).date(),
    "dropoff_time": datetime(2025, 2, 2, 9, 25).date(),
    "fare": 18.75,
    "pickup_location": "Suburbs",
    "dropoff_location": "City Center",
    "status": "completed",
    "pickup_day": datetime(2025, 2, 2).date(),
}

# Append to existing DataFrame
df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)

# convert
table = pa.Table.from_pandas(df, schema=schema, preserve_index=False)

In [54]:
iceberg_table = catalog.load_table("rideshare.rides")
iceberg_table.append(table)

print("🧾 Data without tip_amount appended to Iceberg table.")

🧾 Data without tip_amount appended to Iceberg table.


In [ ]:
with starrocks_cursor() as cursor:
    cursor.execute("SELECT * FROM demo.rideshare.rides;")
    print(cursor.fetchall())